In [164]:
import pandas as pd

ad_clicks = pd.read_csv(r'E:\Jupyter Projects\Ad_Clicks_AB_Testing\ad_clicks.txt')

def display_as_percentage(val):
  return '{:.1}%'.format(val * 100)


## Examining the First Few Rows of ad_clicks

In [165]:
print(ad_clicks.head(5))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


##  Which Ad Platform is Getting the Company the Most Views?
How many views (i.e., rows of the table) came from each 'utm_source'?

In [166]:
ad_clicks\
    .groupby('utm_source')\
    .user_id.count()\
    .reset_index()

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


## Create New Column called 'is_click'
If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed.
Create a new column called 'is_click', which is True if 'ad_click_timestamp' is not null and False otherwise.
 

In [167]:
ad_clicks['is_click'] = ad_clicks\
    .ad_click_timestamp\
    .notnull()
print(ad_clicks)

                                   user_id utm_source            day  \
0     008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday   
1     009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook     7 - Sunday   
2     00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday   
3     011adc64-0f44-4fd9-a0bb-f1506d2ad439     google    2 - Tuesday   
4     012137e6-7ae7-4649-af68-205b4702169c   facebook     7 - Sunday   
...                                    ...        ...            ...   
1649  fe8b5236-78f6-4192-9da6-a76bba67cfe6    twitter     7 - Sunday   
1650  fed3db6d-8c92-40e3-a4fb-1fb9d7337eb1   facebook     5 - Friday   
1651  ff3a22ff-521c-478c-87ca-7dc7b8f34372    twitter  3 - Wednesday   
1652  ff3af0d6-b092-4c4d-9f2e-2bdd8f7c0732     google     1 - Monday   
1653  ffdfe7ec-0c74-4623-8d90-d95d80f1ba34   facebook   6 - Saturday   

     ad_click_timestamp experimental_group  is_click  
0                  7:18                  A      True  
1                   NaN  

## Percentage of People who Clicked on ads from Each Source
We want to know the percent of people who clicked on ads from each utm_source.
Start by grouping by utm_source and is_click and counting the number of user_id's in each of those groups. Save your answer to the variable clicks_by_source.

In [168]:
clicks_by_source = ad_clicks\
  .groupby(['utm_source', 'is_click'])\
  .user_id.count()\
  .reset_index()
 
print(clicks_by_source)

  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66


## Create Clicks Pivot Table 
Now let’s pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id.
Save your results to the variable clicks_pivot.

In [169]:
clicks_pivot = clicks_by_source\
  .pivot(
   columns='is_click',
   index='utm_source',
   values='user_id'
  ).reset_index()
print(clicks_pivot)

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66


## Adding Percent Clicked Column in Clicks Pivot Table
Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who clicked on the ad from each utm_source.
Was there a difference in click rates for each source?


In [170]:
clicks_pivot['percent_clicked'] = clicks_pivot[True] /(clicks_pivot[True] + 
clicks_pivot[False])
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80         0.313725
1          facebook    324   180         0.357143
2            google    441   239         0.351471
3           twitter    149    66         0.306977


## Number of People Showed Each Ad
The column experimental_group tells us whether the user was shown Ad A or Ad B.
Were approximately the same number of people shown both adds?

In [171]:
print(ad_clicks\
  .groupby('experimental_group').user_id\
  .count()\
  .reset_index()
)

  experimental_group  user_id
0                  A      827
1                  B      827


## Number of Users that Clicked on Each Ad
Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [178]:
ad_clicks_pivot = ad_clicks\
  .groupby(['experimental_group', 'is_click']).user_id\
  .count()\
  .reset_index()\
  .pivot(
    index = 'experimental_group',
    columns = 'is_click',
    values = 'user_id'
  )\
  .reset_index()

ad_clicks_pivot['percent_clicked']= ad_clicks_pivot[True]\
    /(ad_clicks_pivot[True] + ad_clicks_pivot[False]) 
print(ad_clicks_pivot)

is_click experimental_group  False  True  percent_clicked
0                         A    517   310         0.374849
1                         B    572   255         0.308343


## Does the Number of Clicks on Each Ad Change Throughout the Week? ##
Start by creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.

In [179]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

## Percentage of Users Who Clicked on the Ad by Day
For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day.


## Ad A:

In [180]:
a_clicks_pivot = a_clicks\
.groupby(['is_click', 'day']).user_id\
.count()\
.reset_index()\
.pivot(
  index = 'day',
  columns = 'is_click',
  values = 'user_id'
)\
.reset_index()\
 
a_clicks_pivot['percent_clicked'] = a_clicks_pivot[True] / (a_clicks_pivot[True] + a_clicks_pivot[False]) 
 
print(a_clicks_pivot)

is_click            day  False  True  percent_clicked
0            1 - Monday     70    43         0.380531
1           2 - Tuesday     76    43         0.361345
2         3 - Wednesday     86    38         0.306452
3          4 - Thursday     69    47         0.405172
4            5 - Friday     77    51         0.398438
5          6 - Saturday     73    45         0.381356
6            7 - Sunday     66    43         0.394495


## Ad B:

In [181]:
b_clicks_pivot = b_clicks\
  .groupby(['is_click', 'day']).user_id\
  .count()\
  .reset_index()\
  .pivot(
    index = 'day',
    columns = 'is_click',
    values = 'user_id'
  )\
  .reset_index()
 
b_clicks_pivot['percent_clicked'] = b_clicks_pivot[True] / (b_clicks_pivot[True] + b_clicks_pivot[False])
 
print(b_clicks_pivot)


is_click            day  False  True  percent_clicked
0            1 - Monday     81    32         0.283186
1           2 - Tuesday     74    45         0.378151
2         3 - Wednesday     89    35         0.282258
3          4 - Thursday     87    29         0.250000
4            5 - Friday     90    38         0.296875
5          6 - Saturday     76    42         0.355932
6            7 - Sunday     75    34         0.311927


## Conclusion
Based on our calculations, users clicked on Ad A more than Ad B on each day of the week except for Tuesday.